In [5]:
#this script will check if our data is acceptable for use (so far it checks if the data is comprised of only zeros and ones)

from PIL import Image
import numpy as np

# Load the TIFF file
tiff_path = r'/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset027_dafnatest2406/labelsTs/image_001.tif'
image = Image.open(tiff_path)

# Convert the image to a numpy array
image_array = np.array(image)

# Check the unique values in the image
unique_values = np.unique(image_array)
print("Unique values in the TIFF file:", unique_values)

# Check if only 0s and 1s are present
if np.array_equal(unique_values, [0,1]):
    print("The TIFF file contains only 0s and 1s.")
else:
    print("The TIFF file contains values other than 0s and 1s.")
    
    # Check the bit depth
print("True label mode:", image.mode)
print("True label dtype:", np.array(image).dtype)


Unique values in the TIFF file: [0 1]
The TIFF file contains only 0s and 1s.
True label mode: I;16
True label dtype: uint16


In [14]:
import os
import numpy as np

def load_image(image_path):
    # Load image using PIL and convert to numpy array
    image = Image.open(image_path)
    image_array = np.array(image)
    return image_array

def calculate_similarity_metrics(image1, image2):
    # Convert images to numpy arrays if they are not already
    image1 = np.array(image1)
    image2 = np.array(image2)
    
    # Ensure images are of the same shape
    if image1.shape != image2.shape:
        raise ValueError("Images must have the same dimensions.")
    
    # Calculate confusion matrix values
    tp = np.sum(np.logical_and(image1 == 1, image2 == 1))  # True Positives
    tn = np.sum(np.logical_and(image1 == 0, image2 == 0))  # True Negatives
    fp = np.sum(np.logical_and(image1 == 1, image2 == 0))  # False Positives
    fn = np.sum(np.logical_and(image1 == 0, image2 == 1))  # False Negatives
    
    # Calculate metrics
    similarity_percentage = (tp + tn) / image1.size * 100  # Overall similarity percentage
    precision = tp / (tp + fp) if tp + fp > 0 else 0.0  # Precision
    recall = tp / (tp + fn) if tp + fn > 0 else 0.0  # Recall
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0.0  # F1 score
    
    # Return metrics
    return {
        'similarity_percentage': similarity_percentage,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'confusion_matrix': {
            'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn
        }
    }

def compare_images_in_folders(folder1, folder2):
    # Get list of files in each folder
    files1 = sorted(os.listdir(folder1))
    files2 = sorted(os.listdir(folder2))
    
    # Check if number of files match
    if len(files1) != len(files2):
        raise ValueError("Number of files in the two folders do not match.")
    
    results = []
    
    # Iterate through files and compare corresponding images
    for file1, file2 in zip(files1, files2):
        if file1 != file2:
            raise ValueError("Files in the two folders do not match.")
        
        # Construct full file paths
        image_path1 = os.path.join(folder1, file1)
        image_path2 = os.path.join(folder2, file2)
        
        # Load images
        image1 = load_image(image_path1)
        image2 = load_image(image_path2)
        
        # Calculate similarity metrics
        metrics = calculate_similarity_metrics(image1, image2)
        
        # Add results to list
        results.append({
            'file': file1,
            'metrics': metrics
        })
    
    return results

# Example usage:
if __name__ == "__main__":
    
    gt_folder = r'/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset027_dafnatest2406/labelsTs' #change this
    results_folder = '/argusdata/users/undergrads/ItamarAndDafna/nnUNet/nnUNet_results/Dataset027_dafnatest2406/tst24.6' #change this
    
    results = compare_images_in_folders(gt_folder, results_folder)

    for result in results:
        print(f"File: {result['file']}")
        print(f"Similarity percentage: {result['metrics']['similarity_percentage']:.2f}%")
        print(f"Precision: {result['metrics']['precision']:.2f}")
        print(f"Recall: {result['metrics']['recall']:.2f}")
        print(f"F1 Score: {result['metrics']['f1_score']:.2f}")
        print(f"Confusion Matrix: TP={result['metrics']['confusion_matrix']['tp']}, TN={result['metrics']['confusion_matrix']['tn']}, FP={result['metrics']['confusion_matrix']['fp']}, FN={result['metrics']['confusion_matrix']['fn']}")
        print()


File: image_001.tif
Similarity percentage: 99.30%
Precision: 0.99
Recall: 0.88
F1 Score: 0.94
Confusion Matrix: TP=6121, TN=113605, FP=44, FN=805

File: image_002.tif
Similarity percentage: 99.58%
Precision: 0.95
Recall: 0.97
F1 Score: 0.96
Confusion Matrix: TP=6213, TN=113854, FP=333, FN=175

File: image_003.tif
Similarity percentage: 99.69%
Precision: 0.95
Recall: 0.98
F1 Score: 0.97
Confusion Matrix: TP=5137, TN=115067, FP=277, FN=94

File: image_004.tif
Similarity percentage: 99.64%
Precision: 0.99
Recall: 0.97
F1 Score: 0.98
Confusion Matrix: TP=9790, TN=110355, FP=129, FN=301

File: image_005.tif
Similarity percentage: 99.41%
Precision: 0.96
Recall: 0.99
F1 Score: 0.97
Confusion Matrix: TP=12035, TN=107824, FP=534, FN=182

File: image_006.tif
Similarity percentage: 99.54%
Precision: 0.97
Recall: 0.99
F1 Score: 0.98
Confusion Matrix: TP=14147, TN=105871, FP=398, FN=159

File: image_007.tif
Similarity percentage: 99.43%
Precision: 0.97
Recall: 0.98
F1 Score: 0.98
Confusion Matrix: 